# Lgbm CV

In [1]:
import pandas as pd 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import gc

In [2]:
with pd.HDFStore('../data/feat/data.h5') as store:
    print(store.keys())
    X_train = store['X_train']
    X_cv = store['X_cv']
    y_train = store['y_train']
    y_cv = store['y_cv']
    X_test = store['X_test']

['/X_cv', '/X_test', '/X_train', '/y_cv', '/y_train']


In [3]:
y_train = y_train.clip(0,20)
y_cv = y_cv.clip(0,20)

In [9]:
params = {
    'bossting_type':'gbdt',
    'objective': 'regression',
    'learning_rate' : 0.5,
    'metric': 'rmse',        
    'max_depth' : 12,
    'num_leaves': 64,
    'feature_fraction': 0.6203,
    'bagging_fraction' : 0.9567,
    'reg_lambda' : 0.3886,
    'gamma' : 0.3962
    
}

In [13]:
# specify your configurations as a dict
# params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'regression',
#     'metric': {'l2', 'rmse'},
#     'num_leaves': 31,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'verbose': 0
# }

In [10]:
lgb_train = lgb.Dataset(X_train,y_train)
# lgb_cv = lgb.Dataset(X_cv,y_cv,reference=lgb_train)
gbm_cv = lgb.cv(params,lgb_train,early_stopping_rounds=5,verbose_eval=True)
# lgb_cv = lgb.Dataset(X_cv,y_cv,reference=lgb_train)
# lgb.cv(params,lgb_train,nfold=5,early_stopping_rounds=5)

[1]	cv_agg's rmse: 1.03234 + 0.00143198
[2]	cv_agg's rmse: 0.969869 + 0.00217628
[3]	cv_agg's rmse: 0.94568 + 0.00212529
[4]	cv_agg's rmse: 0.932161 + 0.00219261
[5]	cv_agg's rmse: 0.92496 + 0.00248139
[6]	cv_agg's rmse: 0.91999 + 0.00288447
[7]	cv_agg's rmse: 0.91606 + 0.00292988
[8]	cv_agg's rmse: 0.912338 + 0.00339231
[9]	cv_agg's rmse: 0.909737 + 0.00295088
[10]	cv_agg's rmse: 0.907743 + 0.00264986
[11]	cv_agg's rmse: 0.905281 + 0.00271093
[12]	cv_agg's rmse: 0.903528 + 0.00251146
[13]	cv_agg's rmse: 0.900599 + 0.00274028
[14]	cv_agg's rmse: 0.898606 + 0.00281831
[15]	cv_agg's rmse: 0.896049 + 0.00251786
[16]	cv_agg's rmse: 0.894117 + 0.00200885
[17]	cv_agg's rmse: 0.891856 + 0.00192826
[18]	cv_agg's rmse: 0.889589 + 0.00151304
[19]	cv_agg's rmse: 0.886479 + 0.00324279
[20]	cv_agg's rmse: 0.885476 + 0.00329226
[21]	cv_agg's rmse: 0.884688 + 0.00351945
[22]	cv_agg's rmse: 0.884 + 0.0033044
[23]	cv_agg's rmse: 0.882579 + 0.0033812
[24]	cv_agg's rmse: 0.882037 + 0.0031244
[25]	cv_agg'

In [12]:
gc.collect()

0

In [13]:
gbm_cv

{'rmse-mean': [1.0323381721608214,
  0.96986926719416322,
  0.94568036585486515,
  0.93216103030699848,
  0.92496029162308024,
  0.91999034231293031,
  0.91606042745827043,
  0.91233804952695652,
  0.90973680437232818,
  0.90774318265118903,
  0.90528102143619105,
  0.90352834988947561,
  0.90059925149454811,
  0.89860582904161357,
  0.89604894591197615,
  0.89411724656026281,
  0.89185592138487579,
  0.88958898904249417,
  0.88647949168224061,
  0.88547550760346749,
  0.88468781195352442,
  0.88399973334494741,
  0.88257942989701699,
  0.88203680904850668,
  0.88075385169054576,
  0.87974546278586607,
  0.87861323981317929,
  0.87763999667357484,
  0.87685597770751955,
  0.87574756411099808,
  0.87487981170824691,
  0.87450659036740785,
  0.87366356665092071,
  0.87326665483544252,
  0.87218910670782113,
  0.87071539408735588,
  0.86994109757767146,
  0.86955280108706978,
  0.86906921304653584,
  0.86837614740944091,
  0.86772251466890249,
  0.86717486832851731,
  0.86678444189720238,

In [15]:
gbm_cv['rmse-mean'][-1]

0.84225748317093974

# Hyperopt

In [19]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error

In [17]:
def lgbm_objective(params):
    ## stolen from :https://github.com/hyperopt/hyperopt/issues/357
    lgb_train = lgb.Dataset(X_train,y_train)
#     lgb_cv = lgb.Dataset(X_cv,y_cv,reference=lgb_train)
    params_set = {
        'bossting_type':'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate' : 0.5,
        'max_depth' : int(params['max_depth']),
        'num_leaves': int(params['num_leaves']),
        'feature_fraction': '{:.3f}'.format(params['feature_fraction']),
        'bagging_fraction' : '{:.3f}'.format(params['bagging_fraction']),
        'reg_lambda' : '{:.3f}'.format(params['reg_lambda']),
        'gamma' : '{:.3f}'.format(params['gamma'])        
    }
    cv = lgb.cv(params_set,
                lgb_train,
                early_stopping_rounds=5,
                verbose_eval=False)

    print('params:{}'.format(params))
    rmse_best_mean = cv['rmse-mean'][-1]
    rmse_best_std = cv['rmse-std'][-1]
    print("rmse_mean_best:{:.5f} + {:.5f}".format(rmse_best,rmse_best_std))
    return {'loss':rmse, 'status': STATUS_OK }

In [ ]:
lgbm_space = { 
    'max_depth' : hp.quniform("max_depth", 4, 16, 1),
    'num_leaves': hp.quniform('num_leaves', 8, 128, 2),
    'feature_fraction': hp.uniform('feature_fraction', 0.3, 1.0),
    'bagging_fraction': hp.uniform ('bagging_fraction', 0.7, 1),
    'reg_lambda': hp.uniform('reg_lambda',0,1),
    'gamma' : hp.uniform('gamma', 0.1,0.5),
}

trials = Trials()
lgbm_best = fmin(fn=lgbm_objective,
            space=lgbm_space,
            algo=tpe.suggest,
            trials = trials,
            max_evals=100)

KeyboardInterrupt: 